<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=170 style="padding: 10px"> 
<b>Little Demo: Raw images from ObsTAP</b> <br>
Contact author(s): Melissa Graham <br>
Last verified to run: 2024-07-20 <br>
LSST Science Pipelines version: Weekly 2024_16 <br>
Container Size: medium

## 1. Introduction

The simulted images for Rubin's Data Preview 0 (DP0) are available via both the
[data butler](https://pipelines.lsst.io/modules/lsst.daf.butler/index.html)
and the TAP (Table Access Protocol) service.

Use of the data butler to query and retrieve images
is covered in other tutorials, and is generally the recommended
way to access DP0 images while working within the Notebook Aspect of the 
Rubin Science Platform, especially if any image reprocessing is being done.

The Portal Aspect of the Rubin Science Platform offers a graphical user interface to the TAP service,
and images can be displayed interactively in the Portal.
No code experience is necessary to use the Portal.

However, raw image access via the TAP service is also possible.
This tutorial demonstrates how to retrieve and display raw images in a Notebook.

### 1.1. Import packages

Import packages a few standard python packages,
[lsst packages](https://pipelines.lsst.io/) for image display and data access,
[astropy packages](https://www.astropy.org/) for working with images,
and a module from the [pvyo.dal package](https://pyvo.readthedocs.io/en/latest/dal/index.html)
for data access.

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from urllib.request import urlretrieve

import lsst.afw.display as afwDisplay
import lsst.geom as geom
from lsst.rsp import get_tap_service
from lsst.afw.image import ExposureF

import astropy.visualization as vis
from astropy.io import fits
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
from astropy import units as u

import warnings
from astropy.wcs import FITSFixedWarning

from pyvo.dal.adhoc import DatalinkResults

### 1.2. Define functions and paramters

Instantiate the RSP's TAP service.

In [ ]:
service = get_tap_service("tap")

Define `auth_session` in order to authorize the TAP service to
access LSST images via Datalink in Section 4.

In [ ]:
auth_session = service._session

Define a point near the center of the Data Preview 0 area with
coordintes Right Ascension (RA) = 62 degrees
and Declination (Dec) = -37 degrees.
Use several different formats for the different use cases
throughout this tutorial

 * `target_ra`, `target_dec` : floats, to be used in calculations
 * `target_str_ra`, `target_str_dec` : strings, to be inserted into query statements
 * `targetPoint` : a `SpherePoint` object from the LSST Science Pipelines' `geom` package
 * `targetCoord` : an astropy `SkyCoord` class

In [ ]:
target_ra = 62.0
target_dec = -37.0
target_str_ra = '62.0'
target_str_dec = '-37.0'
targetPoint = geom.SpherePoint(target_ra*geom.degrees,
                               target_dec*geom.degrees)
targetCoord = SkyCoord(ra=target_ra*u.degree,
                       dec=target_dec*u.degree,
                       frame='icrs')

Define a function `detsec_to_x1x2y1y2` to be used in Section 4.3. 

In [ ]:
def detsec_to_x1x2y1y2(detsec: str):
    """Convert the DETSEC from a FITS header into the
    start and end pixels in x and y.

    Parameters
    ----------
    detsec: `str`
        String formatted as detector section, e.g.,:
        "[509:1,1:2000]" or "[1018:510,1:2000]"

    Returns
    -------
    x1: `int`
        The start x-pixel.
    x2: `int`
        The end x-pixel.
    y1: `int`
        The start y-pixel.
    y2: `int`
        The end y-pixel.
    """
    temp1 = detsec.strip('[').strip(']')
    temp2 = temp1.split(',')
    temp3 = temp2[0].split(':')
    temp4 = temp2[1].split(':')
    x1 = int(temp3[0])
    x2 = int(temp3[1])
    y1 = int(temp4[0])
    y2 = int(temp4[1])
    return x1, x2, y1, y2

## 2. Raw Images

Follow a similar series of steps as in Section 4.1 to retrieve
the pixel data for raw images.

### 2.1. Query and retrieve

One difference between here and Section 4.1 is to constrain the
`obs_id` parameter to be the same as the `calexp`, in order
to obtain the `raw` version of the `calexp` investigated above.

Define the query, submit the job, and retrieve the results.

In [ ]:
use_obs_id = '214437-R20_S20'

In [ ]:
query = "SELECT * FROM ivoa.ObsCore "\
        "WHERE CONTAINS(POINT('ICRS', " + \
        target_str_ra + ", " + target_str_dec + "), s_region) = 1 "\
        "AND lsst_band = 'i' "\
        "AND calib_level = 1 "\
        "AND obs_id = '" + use_obs_id + "'"
print(query)

In [ ]:
job = service.submit_job(query)
job.run()
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)

In [ ]:
results = job.fetch_result()
print(len(results))

Take the first (only!) row and print the same metadata as was printed for the `calexp`.

In [ ]:
results_0 = results[0]

In [ ]:
print(results_0['dataproduct_subtype'])
print(results_0['lsst_ccdvisitid'])
print(results_0['lsst_visit'], results_0['lsst_detector'])
print('%7.4f %8.4f' % (results_0['s_ra'], results_0['s_dec']))
print(results_0['lsst_band'])
print(results_0['t_min'])
print(results_0['s_region'])
print(results_0['obs_id'])

Above, notice that for the `raw` image, the `ccdvisitid` and `visit` are not defined
(this is a temporary, DP0-era issue).
Also notice that the `raw` image search results are sorted differently from the `calexp`, and 
this is a different image (different date, different observation id).

Use `Datalink` to obtain the `image_url`.

In [ ]:
dl_results = DatalinkResults.from_result_url(results_0['access_url'],
                                             session=auth_session)
image_url = dl_results['access_url'][0]
print(image_url)

Print header componenents.

In [ ]:
hdulist = fits.open(image_url)
for hdu in hdulist:
    print(hdu.name)

The above shows that `raw` images (which are one of the LSST Science Camera's 189 detectors) are divided into 16 segments (one for each of the detector's amplifiers).

**Optional:** print the primary header.

In [ ]:
# hdulist[0].header

### 2.2. Display one segment

Display one segment with `matplotlib`.

The following cell will return a few error messages about `astropy` changing
some of the header keywords. It is ok to ignore these for now, and they
will be fixed as the LSST-output FITS files continue to evolve.

To suppress the output of these warnings, use:
```
import warnings
from astropy.wcs import FITSFixedWarning
warnings.filterwarnings('ignore', category=FITSFixedWarning)
```

Get image header and data for the first segment.

In [ ]:
img_hdr = hdulist[1].header
img_wcs = WCS(img_hdr)
img_seg = hdulist[1].name
img_data = hdulist[1].data

**Option** to view the segment header.

In [ ]:
# img_hdr

In [ ]:
zscale = vis.ZScaleInterval()
zlimits = zscale.get_limits(img_data)
print(zlimits)

In [ ]:
fig, ax = plt.subplots(1, figsize=(3, 5))
plt.subplot(projection=img_wcs)
plt.imshow(img_data, cmap='grey',
           vmin=zlimits[0], vmax=zlimits[1])
plt.axis('on')
plt.grid(color='white', ls='solid')
ax.set_xticks([])
ax.set_yticks([])
plt.xlabel('X')
plt.ylabel('Y')
plt.title(img_seg)
plt.show()

Figure 4: One of the segments of the `raw` image, plotted in the same manner as the `calexp` in Section 4.1.5.

What's with the black pixels? They represent the image's bias (a pedestal level of counts added by the amplifier).
In the header, `NAXIS` and `NAXIS2` represent the shape of the image array, and
`DATASEC` provides the dimensions of the pixels with flux.
Note that the `numpy.shape` is (rows, columns), whereas for the FITS file the (axis 1, axis 2) is (columns, rows).

In [ ]:
print(img_hdr['NAXIS1'], img_hdr['NAXIS2'])
print(np.shape(img_data))
print(img_hdr['DATASEC'])

The overscan (bias) pixels are in columns 1, 2, 3, and 513 through 544, and in rows 2001 through 2048.

Compare the mean flux of columns 2000 (flux) and 2001 (overscan).
Index the image array with N-1 because python indexes from 0 whereas FITS indexes from 1.
The bias level is ~1000 counts.

In [ ]:
print('mean counts in column 2000: %7.2f,  column 2001: %7.2f' %
      (np.mean(img_data[1999]), np.mean(img_data[2000])))

### 2.3. Display mosaic

Now that the warnings have been seen once, igore them going forward.

In [ ]:
warnings.filterwarnings('ignore', category=FITSFixedWarning)

In [ ]:
h_indices = np.arange(16, dtype='int') + 1
print(h_indices)

Print the boundary information for each segment.

 * `CRVAL` is the reference coordinate for the mosaic.
 * `CRPIX` are the pixel values relative to the reference.
 * `DATASEC` is the chip pixels that have data.
 * `DETSEC` defines where the segement is in the mosaic.

In [ ]:
for h in h_indices:
    hdr = hdulist[h].header
    wcs = WCS(hdr)
    print('%2i %10s %6s   %5i %5i  %8.4f %8.4f %15s %15s' %
          (h, hdr['EXTNAME'],
           wcs.footprint_contains(targetCoord),
           np.floor(hdr['CRPIX1']), np.floor(hdr['CRPIX2']),
           hdr['CRVAL1'], hdr['CRVAL2'],
           hdr['DATASEC'], hdr['DETSEC']))

Show the segment boundaries in a plot.

Use [qualitative colormap](https://matplotlib.org/stable/users/explain/colors/colormaps.html) `tab20c`.

In [ ]:
cmap = plt.colormaps["tab20c"]

In [ ]:
fig = plt.figure(figsize=(6, 6))
for h in h_indices:
    hdr = hdulist[h].header
    ext = hdr['EXTNAME']
    seg = ext[7:9]
    x1, x2, y1, y2 = detsec_to_x1x2y1y2(hdr['DETSEC'])
    xvals = [x1, x2, x2, x1, x1]
    yvals = [y1, y1, y2, y2, y1]
    plt.plot(xvals, yvals, lw=1, color=cmap(h))
    plt.plot(x1, y1, 'o', ms=h+3, color='None', mec=cmap(h))
    plt.plot(x2, y2, 's', ms=h+3, color='None', mec=cmap(h))
    plt.text(x1-300, y1, seg, color=cmap(h))
plt.xlabel('DETSEC X')
plt.ylabel('DETSEC Y')
plt.title('Segment Map')
plt.show()

Figure 5: Boundaries of the 16 segments of the raw image, each in a different color.
Squares and circle of ascending size mark the `DETSEC` corners.

Create array `mosaic` to hold all segements.

In [ ]:
mosaic = np.zeros((4000, 4072), dtype='float')
print(np.shape(mosaic))

Segment rows 1:2000 indexed at 0.

Segment columns 4:512 indexed at 0.

In [ ]:
seg_row = np.arange(2000, dtype='int')
seg_col = np.arange(509, dtype='int') + 3

Paste each segment into the `mosaic`.

In [ ]:
for h in h_indices:
    hdr = hdulist[h].header
    x1, x2, y1, y2 = detsec_to_x1x2y1y2(hdr['DETSEC'])
    if y2 < y1:
        mos_row = np.arange(start=y1-1, stop=y2-2, step=-1)
    else:
        mos_row = np.arange(start=y1-1, stop=y2, step=1)
    if x2 < x1:
        mos_col = np.arange(start=x1-1, stop=x2-2, step=-1)
    else:
        mos_col = np.arange(start=x1-1, stop=x2, step=1)

    img_data = np.asarray(hdulist[h].data, dtype='float')
    for y, n in zip(seg_row, mos_row):
        for x, m in zip(seg_col, mos_col):
            mosaic[n, m] = img_data[y, x]

Get header, WCS, and define scale limits for image display.

In [ ]:
img_hdr = hdulist[0].header
img_wcs = WCS(img_hdr)
zlimits = zscale.get_limits(mosaic)
print(zlimits)

**Note:** the header WCS is for a FITS-based origin and row-col,
so when the pixels come out of the `img_wcs` here they're row-col (y,x) not col-row (x,y),
so there's a flip that has to happen when plotting col-row.

In [ ]:
fig, ax = plt.subplots(1, figsize=(6, 6))
plt.subplot(projection=img_wcs)
plt.imshow(mosaic, cmap='grey',
           vmin=2250, vmax=2683)

cra = 61.9699 * u.degree
cdec = -36.8508 * u.degree
offset = (90. / 3600.) * u.degree
cPix = img_wcs.world_to_pixel(SkyCoord(cra, cdec, frame='icrs'))
oraPix = img_wcs.world_to_pixel(SkyCoord(cra + offset, cdec, frame='icrs'))
odecPix = img_wcs.world_to_pixel(SkyCoord(cra, cdec + offset, frame='icrs'))
plt.plot([cPix[1], oraPix[1]], [cPix[0], oraPix[0]], 
         ls='solid', lw=1, color='cyan')
plt.plot([cPix[1], odecPix[1]], [cPix[0], odecPix[0]], 
         ls='solid', lw=1, color='yellow')
plt.text(oraPix[1], oraPix[0], 'E', color='cyan')
plt.text(odecPix[1], odecPix[0], 'N', color='yellow')
del cra, cdec, offset, cPix, oraPix, odecPix

ax.set_xticks([])
ax.set_yticks([])
plt.xlabel('DETSEC X')
plt.ylabel('DETSEC Y')
plt.axis('on')
plt.grid(color='white', ls='solid')
plt.title('Mosaic with 1,1 at lower left')
plt.show()

Figure 6: A mosaic of the raw image, with a compass showing north and east in the middle.

Save this file.

In [ ]:
filename = os.path.join(os.getenv("HOME"), 'dp02_02c_image_raw.fits')
urlretrieve(image_url, filename)